In [1]:
# imports
import requests
import pandas as pd
import os
import json
import csv
import folium

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [2]:
cb_df = pd.read_csv('station_data.csv')
fy_df = pd.read_csv('merged_fsq_df.csv')

In [3]:
# merge data frames
merged_df = cb_df.merge(fy_df, on='station_id')

sorted_df = merged_df.sort_values(by='rating', ascending=False)

# view new data frame
sorted_df.head()


,station_id,Latitude,Longitude,Num_of_Slots,fsq_id,fsq_lat,fsq_long,id,name,popularity,rating,address
292,13021bba7bbeca1ccd8726ca17da59e6,49.25094,-123.101306,14,51b201d27dd249ae714ba728,49.263877,-123.105307,13003.0,Bar,0.945327,9.3,"15 8th Ave W (btwn Manitoba & Ontario), Vancou..."
285,13021bba7bbeca1ccd8726ca17da59e6,49.25094,-123.101306,14,4c0b1f677e3fc928cdebf482,49.295623,-123.150768,16003.0,Beach,0.989325,9.1,"8501 Stanley Park Dr (at N Lagoon Dr), Vancouv..."
394,13021bba7bbeca1ccd8726ca17da59e6,49.25094,-123.101306,14,50d4bd11e4b0b55bb4b657a3,49.266477,-123.140728,13002.0,Bakery,0.965011,9.0,"2150 Fir St (at W 6th Ave), Vancouver BC V6J 3B5"
163,13021bba7bbeca1ccd8726ca17da59e6,49.25094,-123.101306,14,4b77ba28f964a52064a82ee3,49.279619,-123.101293,13009.0,Cocktail Bar,0.971802,9.0,"135 Keefer St (btwn Columbia & Main), Vancouve..."
286,13021bba7bbeca1ccd8726ca17da59e6,49.25094,-123.101306,14,4bf046d53a002d7fb4eb85a4,49.295495,-123.150919,18075.0,Swimming Pool,0.927823,8.9,"8501 Stanley Park Dr (at N Lagoon Dr), Vancouv..."


In [4]:
# create csv file for our model
sorted_df.to_csv('final_df.csv', index=False)

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [5]:
# number of bikes at each station
file_1 = 'num_slots.csv'
df_1 = pd.read_csv(file_1)

# fsq coords nearest to stations
file_2 = 'merged_fsq_df.csv'
df_2 = pd.read_csv(file_2)

In [6]:
# Create a map of central Vancouver
map = folium.Map(location=[49.2835, -123.1113], zoom_start=14)

In [7]:
# Add markers for the first dataset
for index, row in df_1.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    coords = [latitude, longitude]
    
    # Create a custom icon with the desired color
    icon = folium.Icon(color='red')
    
    # Add the marker with the custom icon to the map
    folium.Marker(coords, popup=f"City Bike Station\nSlots: {row['Num_of_Slots']}", icon=icon).add_to(map)

In [8]:
# Add markers for the second dataset
for index, row in df_2.iterrows():
    latitude = row['fsq_lat']
    longitude = row['fsq_long']
    coords = [latitude, longitude]
    folium.Marker(coords, popup=row['name']).add_to(map)

In [9]:
# Display the map
map

** Legend: City Bike Stations - Red, POI - Blue **

As we can see in the map, whenever there is a heavily dense area with Points of interest—mainly arts and entertainment, dining and drinking, and landmarks and outdoor places—we will see several City Bike stations. If you click on the red markers in areas near the water or in large parks, you will notice a higher number of slots (potentially more bikes) at those stations.

In [10]:
map.save("map.html")

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [11]:
# import 
import sqlite3
from sqlite3 import Error

In [12]:
# create database
connection = sqlite3.connect('cb_project.db')

In [13]:
# create cursor
cursor = connection.cursor()

In [14]:
# create table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS cb_project (
        station_id TEXT,
        Latitude FLOAT,
        Longitude FLOAT,
        Num_of_Slots INTEGER,
        fsq_id TEXT,
        fsq_lat FLOAT,
        fsq_long FLOAT,
        id FLOAT,
        name TEXT,
        popularity FLOAT,
        rating FLOAT,
        address TEXT
    )
""")

In [15]:
connection.commit()

In [16]:
# upload data to table
sorted_df.to_sql('cb_project', connection, if_exists='replace', index=False)

959

Look at the data before and after the join to validate your data.

In [17]:
# check total number of rows
cursor = connection.cursor()
cursor.execute("SELECT COUNT(*) FROM cb_project")
count = cursor.fetchone()[0]
print("Number of rows:", count)

Number of rows: 959


In [18]:
sorted_df.shape

(959, 12)

I validated the data by confirming the number of rows matched. 